**PROBLEM STATEMENT**

Develop a methodology for calculating key performance indicators (KPIs) that relate to the environmental and social issues that are discussed in the CDP survey data. Leverage external data sources and thoroughly discuss the intersection between environmental issues and social issues. Mine information to create automated insight generation demonstrating whether city and corporate ambitions take these factors into account.

# **INTRODUCTION**

This notebook explores the climate hazards faced by cities around the world, the impact on people, and the mitigation plans put in place to deal with the consequences of the risks as disclosed to CDP in the annual surveys.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas.plotting import parallel_coordinates as pc
from ipywidgets import interact, interactive
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.graph_objects import Layout
import warnings
from tabulate import tabulate
from palettable.colorbrewer.qualitative import Pastel1_7
warnings.simplefilter('ignore')

In [ ]:
import ee.mapclient
import os
import folium
import ee
from kaggle_secrets import UserSecretsClient
from google.oauth2.credentials import Credentials
import rasterio as rio
mysecret = "secretz" 
refresh_token = UserSecretsClient().get_secret(mysecret)
credentials = Credentials(None,refresh_token=refresh_token,token_uri=ee.oauth.TOKEN_URI,
        client_id=ee.oauth.CLIENT_ID,client_secret=ee.oauth.CLIENT_SECRET,scopes=ee.oauth.SCOPES)
ee.Initialize(credentials=credentials)

In [ ]:
citid18 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2018_Cities_Disclosing_to_CDP.csv')
citid19 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv')
citid20 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv')
citid = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/Cities_Disclosing_to_CDP_Data_Dictionary.csv')
citires18 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2018_Full_Cities_Dataset.csv')
citires19 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2019_Full_Cities_Dataset.csv')
citires20 = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv')
citires = pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/Full_Cities_Response_Data_Dictionary.csv')
acctorg = dict(zip(citid19['Account Number'], citid19['Organization']))
nox = pd.read_csv('../input/city-no2-emissions/citid19.csv')

In [ ]:
#population
population = citires19[(citires19['Question Number']=='0.5') & (citires19['Column Number']==1)][['Account Number','Response Answer']]
population = population.rename(columns={'Response Answer':'Popn'})
population['Popn'] = population['Popn'].astype('float')
population = population[(population['Popn']>100000) & (population['Popn']<25000000)]

#area
area = citires19[(citires19['Question Number']=='0.6') & (citires19['Column Number']==1)][['Account Number','Response Answer']]

# Climate Commitments

Let us take a look at the different covenants that the cities have signed up for, and the network sizes of the different charters.

In [ ]:
com = citires19[(citires19['Question Number']=='1.1a') & (citires19['Column Number']==1)][['Account Number','Row Number', 'Response Answer']]
com = com.replace('Deadline 2020 - Delivering the 1.5 degree ambition of the Paris Agreement in a resilient, inclusive way', 'Paris Agreement')
com = com.replace('Global Covenant of Mayors for Climate & Energy: Compact of mayors & Individual city committment', 'Global Covenant of Mayors for Climate & Energy')
comtype = citires19[(citires19['Question Number']=='1.1a') & (citires19['Column Number']==2)][['Account Number','Row Number','Response Answer']]
com['Response Answer'] = com['Response Answer'].astype('str')
com['Response Answer'] = com['Response Answer'].apply(lambda x: 'Other' if 'Other' in x else x)
com = com[com['Response Answer']!='nan']
comtype['Response Answer'] = comtype['Response Answer'].astype('str')
comtype['Response Answer'] = comtype['Response Answer'].apply(lambda x: 'Other' if 'Other' in x else x)
comtype = pd.merge(com, comtype, on=['Account Number', 'Row Number'])
comtype = comtype.rename(columns={'Response Answer_x':'Commitment', 'Response Answer_y':'Commit Type'})
comchart = comtype.groupby(['Commitment','Commit Type']).size().unstack('Commit Type', fill_value=0)
comchart = comchart[['Both', 'Adaptation', 'Mitigation','Other','nan']]
comchart = comchart.rename(columns={'nan':'Unspecified'})
comchart['s']=comchart.sum(axis=1)
comchart = comchart.sort_values(by='s')
colz = ['Both', 'Adaptation', 'Mitigation','Other','Unspecified']
ax = comchart[colz].plot.barh(stacked=True, figsize=(15,8), grid=True,
                         color={'Both': '#810F1B', 'Adaptation':'#D2681B', 'Mitigation': '#D2BB1B', 'Unspecified': 'Gray', 'Other':'#98894A'})
ax.xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
ax.yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)
ax.set_xlabel('No of Signatories')
ax.set_title('Overview of Climate Commitments', fontsize=20)
plt.show()

Most cities have committed to both adaptation and mitigation measures, with a higher focus on mitigation.
The **Global Covenant of Mayors for Climate and Energy**, which has the largest number of participating cities, was established in 2016 by merging the Compact of Mayors and the European Union's Covenant of Mayors. It is a global coalition of city leaders addressing climate change by pledging to cut greenhouse gas emissions and prepare for the future impacts of climate change. The Compact highlights cities’ climate impact while measuring their relative risk levels and carbon pollution.The Compact represents a common effort from global city networks C40 Cities Climate Leadership Group (C40), ICLEI, and United Cities and Local Governments (UCLG), as well as UN-Habitat, to unite against climate change.

Cities have identified various climate hazards, as well as the probability and consequences of each type of risk. The different types of risks can be classified into 8 main categories -
1. Extreme Weather (temperatures, precipitation)
2. Water Scarcity
3. Floods and Sea-level Rise
4. Storm and Wind
5. Wild Fire
6. Mass Movement (landslides and avalanches)
7. Biological Hazards (water and air-borne diseases)
8. Chemical Change (Ocean acidificaiton, salt water intrusion)

The total risk of a given hazard can be calculated as -

#   **Total Risk = Likelihood x Impact**

Let us begin by taking a look at how the risks are distributed.

In [ ]:
riskmap = {'Does not currently impact the city':0, 'Do not know':0, 'Low':1, 'Medium Low':2, 'Medium':3, 'Medium High':4, 'High':5}
hazards = citires19[(citires19['Question Number']=='2.1')][['Account Number','Column Number','Row Number','Response Answer']]
hazids = hazards[(hazards['Column Number']==1)]
hazids = hazids.rename(columns={'Response Answer':'Hazard'})
hazids = hazids[~hazids['Hazard'].isna()]
hazids['Hazard']=hazids['Hazard'].astype('str')
hazids['Hazard']=hazids['Hazard'].apply(lambda x: x.split(' >')[0])
hazids['Hazard']=hazids['Hazard'].apply(lambda x: 'Extreme Weather' if 'Extreme' in x else x)
hazids['Hazard']=hazids['Hazard'].replace({'Flood and sea level rise':'Flood & sea level rise'})
probab = hazards[(hazards['Column Number']==3)]
conseq = hazards[(hazards['Column Number']==4)]
timing = hazards[(hazards['Column Number']==8)]
hazids = pd.merge(hazids, probab, on=['Account Number', 'Row Number'] )
hazids = pd.merge(hazids, conseq, on=['Account Number', 'Row Number'] )
hazids = pd.merge(hazids, timing, on=['Account Number', 'Row Number'] )
hazids = hazids.rename(columns={'Response Answer_x':'Likelihood', 'Response Answer_y':'Impact','Response Answer':'Timing'})
hazids['Total Risk'] = hazids['Impact'].map(riskmap)*hazids['Likelihood'].map(riskmap)
soconseq = hazards[(hazards['Column Number']==5)].sort_values(by='Row Number')
soconseq = soconseq.rename(columns={'Response Answer':'Social Impact'})
soconseq = pd.merge(hazids, soconseq, how='left', on=['Account Number','Row Number'])
soconseq['Hazard'] = soconseq['Hazard'].astype('str')
soconseq['Hazard'] = soconseq['Hazard'].apply(lambda x: 'Other' if 'Other' in x else x)
soconseq['Social Impact'] = soconseq['Social Impact'].astype('str')
soconseq['Social Impact'] = soconseq['Social Impact'].apply(lambda x: 'Other' if 'Other' in x else x)
soconseq = soconseq[soconseq['Social Impact']!='nan']
socdic = {'Increased risk to already vulnerable populations': 'Increased risk to vulnerable pop',
         'Increased demand for healthcare services':'Increased demand for healthcare',
         'Increased incidence and prevalence of disease and illness':'Increased disease and illness',
         'Loss of tax base to support public services':'Loss of tax base'}
soconseq['Social Impact'] = soconseq['Social Impact'].replace(socdic)
vulnerable = hazards[(hazards['Column Number']==10)]
vulnerable = pd.merge(vulnerable, hazids, on=['Account Number','Row Number'])
vulnerable['Response Answer'] = vulnerable['Response Answer'].astype('str').apply(lambda x: 'Other' if 'Other' in x else x)
vulnerable['Response Answer'] = vulnerable['Response Answer'].replace('nan', 'Other')

plt.xlabel('Total Risk')
plt.title('Histogram of Risks')
alf=0.3
plt.axvspan(0, 5, facecolor='green', alpha=alf,label='Lo')
plt.axvspan(5, 10, facecolor='#8CBE2C', alpha=alf, label='Md-Lo')
plt.axvspan(10, 15, facecolor='yellow', alpha=alf, label='Med')
plt.axvspan(15, 20, facecolor='orange', alpha=alf, label='Md-Hi')
plt.axvspan(20, 25, facecolor='red', alpha=alf, label='Hi')
plt.hist(hazids['Total Risk'],bins=5,  color='black', alpha=0.5, rwidth=0.8)
plt.xlim(left=0, right=25)
plt.ylim(bottom=0, top=1100)
plt.legend(ncol=5)
plt.grid(True, 'major', linestyle='--', linewidth=.3)
plt.show()

In [ ]:
riskadapt = citires19[(citires19['Question Number']=='3.0') & (citires19['Column Number']==1)][['Account Number', 'Row Number', 'Response Answer']]
riskadaptact = citires19[(citires19['Question Number']=='3.0') & (citires19['Column Number']==2)][['Account Number', 'Row Number', 'Response Answer']]
riskadaptactstat = citires19[(citires19['Question Number']=='3.0') & (citires19['Column Number']==4)][['Account Number', 'Row Number', 'Response Answer']]
riskadapt2 = riskadapt.merge(riskadaptact, on=['Account Number','Row Number'])
riskadapt2 = riskadapt2.merge(riskadaptactstat, on=['Account Number','Row Number'])
riskadapt2['Response Answer_y']=riskadapt2['Response Answer_y'].astype('str')
riskadapt2['Response Answer_y']=riskadapt2['Response Answer_y'].apply(lambda x: 'Other' if 'Other' in x else x)
riskadapt2['Response Answer_x']=riskadapt2['Response Answer_x'].astype('str')
riskadapt2['Response Answer_x']=riskadapt2['Response Answer_x'].apply(lambda x: x.split(' >')[0])
riskadapt2 = riskadapt2.rename(columns={'Response Answer_x':'Hazard','Response Answer_y':'Actions','Response Answer':'Status'})
actdict = {'Public preparedness (including practice exercises/drills)':'Public preparedness',
           'Public preparedness (including practice exercises/drills): Mangrove Planting':'Public preparedness',
           'Shading in public spaces, markets':'Shading in public spaces',
           'Crisis management including warning and evacuation systems: Heat Distribution Mapping':'Crisis mgmt inc warning & evac sys',
           'Crisis management including warning and evacuation systems':'Crisis mgmt inc warning & evac sys',
           'Hazard resistant infrastructure design and construction':'Hazard resistant infrastructure',
          }

riskadapt2['Actions'] = riskadapt2['Actions'].replace(actdict)
riskadapt2['Hazard']=riskadapt2['Hazard'].astype('str')
riskadapt2['Hazard']=riskadapt2['Hazard'].apply(lambda x: x.split(' >')[0])
riskadapt2['Hazard']=riskadapt2['Hazard'].apply(lambda x: 'Extreme Weather' if 'Extreme' in x else x)
riskadapt2['Hazard']=riskadapt2['Hazard'].replace({'Flood and sea level rise':'Flood & sea level rise'})
hazids2 = pd.merge(hazids, riskadapt2, on=['Account Number', 'Hazard'])
timedict = {np.nan:'Unknown', 'Medium-term (2026-2050)':'Medium Term', 'Long-term (after 2050)':'Long Term',
       'Immediately': 'Immediate', 'Short-term (by 2025)':'Short Term'}
hazids2['Timing'] = hazids2['Timing'].map(timedict)
timewise = hazids2.groupby(['Timing','Hazard'])['Total Risk'].sum().reset_index()
timewise = timewise.pivot_table(values=['Total Risk'], index='Timing', columns='Hazard', aggfunc='sum')
timeorder = ['Immediate','Short Term','Medium Term','Long Term']
timewise = timewise.reindex(timeorder)
timewise.columns = timewise.columns.to_flat_index() #timewise.columns.get_level_values(1)
timewise.columns = [x[1] for x in timewise.columns]
timewise = timewise.fillna(0)
timewisestack = timewise.div(timewise.sum(1), axis=0)
hazs = hazids2['Hazard'].unique()
riskcolors = ['#3F5A7D', '#D2BB1B','#99B1D1', '#D2681B', '#673E0D', 'gray','#593567','#356738']
fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].bar(timeorder,timewisestack[hazs[0]], label=hazs[0], color = riskcolors[0])
ax[0].set_title('Relative Proportion of Risks wrt Time Horizon', fontsize=18)
s = timewisestack[hazs[0]]
for i in range(1,8):
    ax[0].bar(timeorder,timewisestack[hazs[i]], label=hazs[i], bottom=s, color=riskcolors[i])
    s = s+timewisestack[hazs[i]]
ax[0].grid(True, 'major', linestyle='--', linewidth=.3)
ax[1].bar(timeorder,timewise[hazs[0]], label=hazs[0], color = riskcolors[0])
for i in range(1,8):
    ax[1].bar(timeorder,timewise[hazs[i]], label=hazs[i], color = riskcolors[i])
ax[1].grid(True, 'major', linestyle='--', linewidth=.3)
ax[1].set_title('Total Risks Identified', fontsize=18)
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.show()

While we have a good understanding of immediate and short-term risks, most cities have not identified any significant long-term risks.
Of the identified risks, while extreme weather poses a large risk in the immediate future, the threat of flood and sea-level rise looms large among the few long-term risks identified.

# Social Impact of Identified Climate Hazards

In [ ]:
socheatmap = soconseq.groupby(['Hazard','Social Impact']).size().unstack('Social Impact', fill_value=0)
fig, ax = plt.subplots(figsize=(15,5))
sns.heatmap(socheatmap, ax=ax)
plt.yticks(rotation=0, fontsize=20)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.xticks(rotation=70, ha='left', fontsize=15)
plt.title('Social Impact of Climate Hazards', fontsize=20)
plt.show()

In [ ]:
nsocheatmap = socheatmap.div(socheatmap.sum(1), axis=0)
fig, ax = plt.subplots(figsize=(15,5))
sns.heatmap(nsocheatmap, ax=ax)
plt.yticks(rotation=0, fontsize=20)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.xticks(rotation=70, ha='left', fontsize=15)
plt.title('Normalized Social Impact of Climate Hazards', fontsize=20)
plt.show()

On a global scale, the greatest impact of extreme weather and other hazards will be on vulnerable populations.

In [ ]:
vulnerability = vulnerable.groupby('Response Answer')['Total Risk'].sum()
names= list(vulnerability.index)
size=list(vulnerability.values)
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(size, labels=names, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('Population Vulnerable to Climate Risks', fontsize=18)
plt.show()

The sub-groups that would be most affect include the elderly, poor, children and youth.

# Risk Management KPIs

Cities have identified various mitigation measures that they have undertaken to deals with the risks of climate change.

In [ ]:
def f(Dataset):
    df = hazids2[hazids2['Hazard']==Dataset]
    figa1 = go.Figure(go.Parcats(dimensions=[{'label':'Hazard', 'values':df['Hazard']},
                                        {'label':'Actions', 'values':df['Actions']},
                                        {'label':'Status', 'values':df['Status']}], 
                            #line={'color': df['Status'], 'colorscale': colorscale},
                               bundlecolors=True))
    figa1 = go.FigureWidget(figa1)
    figa1.update_layout()
    widgets.HBox([figa1])
    figa1.show()
    return Dataset
interact(f, Dataset = widgets.RadioButtons(options=[hazs[i] for i in range(len(hazs))],description='Select:',disabled=False))

In [ ]:
figs = []
catar = 8*[[ 'Scoping','Pre-feasibility study', 'Pre-implementation', 'Implementation', 'Implementation complete', 'Operation', 'Monitoring and reporting']]
catar[7] = [ 'Scoping','Pre-feasibility study', 'Implementation', 'Implementation complete', 'Operation', 'Monitoring and reporting']
hazids2['Total Risk Cat'] = pd.cut(hazids2['Total Risk'], bins=[0,5,10,15,20,25], labels=['Low','Medium Low','Medium','Medium High','High'])
hazids2['Total Risk Cat']=hazids2['Total Risk Cat'].cat.add_categories("Unknown").fillna("Unknown")
hazids2['Status'] = hazids2['Status'].replace('Implementation complete but not in operation','Implementation complete')
riskmap = {'Low':1, 'Medium Low':2, 'Medium':3, 'Medium High':4, 'High':5, 'Unknown':0}
hazids2['Total Risk Catc']=hazids2['Total Risk Cat'].astype('str').map(riskmap)
colorscale =px.colors.diverging.RdYlGn #['#488235','#CCC731','#EBE893','#CC8025','#CC2A25', 'gray']
for i in range(len(hazs)):
    df = hazids2[hazids2['Hazard']==hazs[i]]
    figa = go.Figure(go.Parcats(dimensions=[{'label':'Impact Timing', 'values':df['Timing'],  'categoryorder': 'array', 'categoryarray':['Unknown', 'Immediate','Short Term', 'Medium Term', 'Long Term']},
                                        {'label':'Total Risk', 'values':df['Total Risk Cat'], 'categoryorder': 'array', 'categoryarray':['Unknown', 'Low','Medium Low', 'Medium', 'Medium High', 'High']},
                                        {'label':'Mitigation Status', 'values':df['Status'], 'categoryorder': 'array', 'categoryarray':catar[i]}], 
                            line={'color': df['Total Risk Catc'], 'colorscale': colorscale},
                               bundlecolors=True))
    figa = go.FigureWidget(figa)
    figa.layout.title = hazs[i]
    figa.update_layout(
    margin=dict(l=50, r=100, t=50, b=20),
    paper_bgcolor="#EBEDEF",autosize=False,width=800,height=300)
    figa.show()

For defining a risk management KPI, we need to take into account the magnitude of the present risk, as well as the status of the mitigation actions. Cities that are pro-active and effective in risk management should have implemented mitigation measures that are operating well, and being monitored and reported transparently on a regular basis.
We define a Status Score scale to rate the mitigation measures from 0 to 1, where 0 implies no mitigation actions identified, and 1 is assigned to measures that are the most a city can do - Monitor and Report the implemented mitigation plan.

In [ ]:
Status_scores = {'Unknown':0, 'Scoping':0.1, 'Pre-feasibility study':0.2, 'Pre-implementation':0.25,
                 'Implementation':0.5, 'Implementation complete':0.75, 'Operation':0.9, 'Monitoring and reporting':1}
tix = [*Status_scores.values()]
a = np.array([tix])
figa = plt.figure(figsize=(15, 1))
img = plt.imshow(a, cmap="RdYlGn")
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
cbar = plt.colorbar(orientation="horizontal", cax=cax, ticks=tix)
figa.axes[1].set_xticklabels(list(Status_scores.keys()))
figa.axes[1].xaxis.tick_top()
plt.xticks(rotation=20, ha='left', fontsize=16)
plt.title('Implementation Status Score Scale', fontsize = 20)
plt.show()
print(tabulate(Status_scores.items(), headers=['Status','Status Score'], tablefmt="grid"))

To take into account the magnitude of the risks, each Status Score is weighted by the total risk it covers, so that actions mitigating greater risks score higher than actions mitigating lower risks. This is to ensure that cities that give primacy to high risk items are rewarded commensurately with higher KPI scores.
Taking all the above into account, we arrive at a Risk Management Index as calculated below -

#                 Risk Management Index
# RMI=         $ \frac{\sum_{i=1}^nStatus Score_i*Total Risk_i}{\sum_{i=1}^nTotal Risk_i} $

This is calculated for immediate risks.

To take into account readiness to face future risks, we define a **Risk Preparedness Index**, which uses the same formula as the Risk Management Index, but is calculated for identified future risks. 

In [ ]:
hazids2['Status_score'] = hazids2['Status'].map(Status_scores)
hazids2['Risk Adaptation'] = hazids2['Total Risk']*hazids2['Status_score']
rmi = {}
hazids2imm = hazids2[hazids2['Timing']=='Immediate']
for i in range(len(hazs)):
    df = hazids2imm[hazids2imm['Hazard']==hazs[i]]
    dfacct = df.groupby('Account Number')
    dfr = pd.DataFrame(dfacct['Risk Adaptation'].sum()/dfacct['Total Risk'].sum(), columns=['Risk Adaptation'])   
    dfr['Org'] = dfr.index.map(acctorg)
    rmi[hazs[i]]=dfr.sort_values(by='Risk Adaptation', ascending=False)
    
rpi = {}
hazids2fut = hazids2[hazids2['Timing']!='Immediate']
for i in range(len(hazs)):
    df = hazids2fut[hazids2fut['Hazard']==hazs[i]]
    dfacct = df.groupby('Account Number')
    dfr = pd.DataFrame(dfacct['Risk Adaptation'].sum()/dfacct['Total Risk'].sum(), columns=['Risk Adaptation'])   
    dfr['Org'] = dfr.index.map(acctorg)
    rpi[hazs[i]]=dfr.sort_values(by='Risk Adaptation', ascending=False)
fig, ax = plt.subplots(4,2, figsize=(20,10), constrained_layout=True)
for i in range(len(hazs)):
    ax[int(i/2),i%2].barh(rmi[hazs[i]].head()['Org'], rmi[hazs[i]].head()['Risk Adaptation'], color='#3D6034')
    ax[int(i/2),i%2].invert_yaxis()
    ax[int(i/2),i%2].set_title(hazs[i])
    ax[int(i/2),i%2].set_xlim(right=1)
    ax[int(i/2),i%2].xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
    ax[int(i/2),i%2].yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)
    ax[int(i/2),i%2].tick_params(axis='y', which='major', labelsize=15)


#plt.tight_layout()
fig.suptitle('Top Cities in Risk Management', fontsize=25)
plt.show()

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(20,10), constrained_layout=True)
for i in range(len(hazs)):
    ax[int(i/2),i%2].barh(rpi[hazs[i]].head()['Org'], rpi[hazs[i]].head()['Risk Adaptation'], color='#3D6034')
    ax[int(i/2),i%2].invert_yaxis()
    ax[int(i/2),i%2].set_title(hazs[i])
    ax[int(i/2),i%2].set_xlim(right=1)
    ax[int(i/2),i%2].xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
    ax[int(i/2),i%2].yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)

#plt.tight_layout()
fig.suptitle('Top Cities in Risk Preparedness', fontsize=25)
plt.show()

# Emissions KPIs

According to the Global Protocol for Community-Scale Greenhouse Gas Emission Inventories (GPC), cities are responsible for more than 70 percent of global energy-related carbon dioxide emissions, and present the single greatest opportunity for tackling climate change. It defines a standard protocol for measuring GHG emissions.

Emissions are classified based on scope, as well as the source. The scope categorizes emissions into 3 classes based on their generation and usage with respect to city boundaries. The major sources of include Stationary Energy generation, Transportation, Waste, Industry, and Agriculture.

In [ ]:
emcats = ['Stat_energy-1', 'Stat_energy-2', 'Stat_energy-3', 'Stat_energy2grid-1','Transport-1', 'Transport-2', 'Transport-3', 'Waste-1', 'Waste-3', 'Wasteout-1',
         'Industry-1', 'Agri-1', 'Total-1', 'Total-2', 'Total-3', 'Totalbasic', 'Totalbasic+']
emissions, citicount = {}, {}
for i in range(len(emcats)):
    emi = citires19[(citires19['Question Number']=='4.6b') & (citires19['Column Number']==1) & (citires19['Row Number']==i+1)]
    emires = emi[emi['Response Answer']!='Question not applicable']
    emissions[emcats[i]]= emires
    citicount[emcats[i]] = emires['Response Answer'].count()
emscope1exgrid = citires19[(citires19['Question Number']=='4.6c') & (citires19['Column Number']==1)]
emscope1ingrid = citires19[(citires19['Question Number']=='4.6c') & (citires19['Column Number']==3)]
emscope1tot = citires19[(citires19['Question Number']=='4.6c') & (citires19['Column Number']==5)]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,4))
ax.xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
ax.yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)
plt.xticks(rotation=70, ha='right')
plt.bar(list(citicount.keys()), list(citicount.values()))
plt.title('Number of Responses Received per Category')
plt.show()

We have emissions declaration data from fewer than 140 cities in all categories.

In [ ]:
my_circle=plt.Circle( (0,0), 0.5, color='white')
x = emcats[:-5]
y = [emissions[cat]['Response Answer'].astype('float').mean() for cat in emcats[:-5]]
plt.gcf().gca().add_artist(my_circle)
colors = Pastel1_7.hex_colors
patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
labels = x
plt.legend(patches, labels, loc='right center', bbox_to_anchor=(1, 1.),fontsize=10)
plt.title('Breakdown of Emissions by Scope and Source')
plt.show()

Energy Generation accounts for over 50% of the total emissions in all cities. A simple KPI for measuring the carbon footprint of a city would be **Carbon Emissions per Capita**. However, as the data per city is not widely available, we can use NOx emissions as a proxy. NOx is emitted along with CO2 in vehicular exhausts, with the NOx/CO2 ratio determined by the Emissions Standard of the country.

The Sentinel-5 Precursor satellite has an onboard sensor - Tropomi (TROPOspheric Monitoring Instrument), which measures NOx. Data is available on [Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2).

NOX CONCENTRATIONS WORLDWIDE

In [ ]:
column = 'tropospheric_NO2_column_number_density'
dataset = "COPERNICUS/S5P/OFFL/L3_NO2"
begin_date = '2019-01-01'; end_date = '2019-12-31'
minval = 0; maxval=.0002
latitude = 60; longitude = 25.9
zoom = 2
vis_params = {'min':minval, 'max':maxval,'palette': ['green', 'cyan', 'blue', 'yellow', 'orange', 'red']}
my_map = folium.Map(location=[latitude,longitude], zoom_start=zoom, height=500)
s5p = ee.ImageCollection(dataset).filterDate(begin_date, end_date)
ime = s5p.select(column).reduce(ee.Reducer.mean())
map_id = ime.getMapId(vis_params)
folium.raster_layers.TileLayer(
    tiles = map_id['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,control = True).add_to(my_map)
display(my_map)

(Note: Slow Loading Graphs)

If the kernel has not been run in a couple of days, the above Earth Engine layer may not show, so below is a representative layer downloaded and saved offline as a [dataset](https://www.kaggle.com/parselt/noxmap).

In [ ]:
im = '../input/noxmap/ime.tif'
band = rio.open(im).read()
band10 = band[0]*10000
band10 = np.where(band10>1,1,band10)
band10 = np.where(np.isnan(band10), 0, band10)
bounds = [[90, -180], [-90, 180]]
m = folium.Map(height=300, width=500)
folium.raster_layers.ImageOverlay(image=band10,colormap=lambda x: (1, 0, 0, x), bounds=bounds).add_to(m)
display(m)

**NOx METHODOLOGY**

The city is assumed to be a square of the same area as reported in the disclosure files. Tropospheric NOx emissions for a city is extracted for the given area, and averaged over all of 2019 in the [City NO2 Emissions](https://www.kaggle.com/parselt/city-no2-emissions) notebook.
**NOx per Capita** is calculated as a standard emissions KPI.

In [ ]:
#nox = pd.read_csv('../input/city-no2-emissions/citid19.csv')
nox = pd.merge(nox, population, on='Account Number')
nox['no2']=nox['no2'].astype('float')
nox['Popn']=nox['Popn'].astype('float')
nox['Nox_percapita'] = nox['no2']/nox['Popn']*10**10
nox['Org']= nox['Account Number'].map(acctorg)
nox = nox[nox['Nox_percapita']>0]
nox = nox.sort_values(by='Nox_percapita', ascending=False)
fig, ax = plt.subplots(1,2, figsize=(15,5), constrained_layout=True)
ax[0].barh(nox.head(10)['Org'], nox.head(10)['Nox_percapita'])
ax[0].invert_yaxis()
ax[1].barh(nox.tail(10)['Org'], nox.tail(10)['Nox_percapita'])
ax[0].set_title('Highest Nox per Capita')
ax[1].set_title('Lowest Nox per Capita')
for ax in [ax[0], ax[1]]:
    ax.xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
    ax.yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)
    ax.tick_params(axis='y', which='major', labelsize=15)

In [ ]:
emitreduct = citires19[(citires19['Question Number']=='5.0a')]
emitreductsec = emitreduct[emitreduct['Column Number']==1].rename(columns={'Response Answer':'Sector'})
emitbaseyr = emitreduct[emitreduct['Column Number']==4].rename(columns={'Response Answer':'Base Year'})
emitbase = emitreduct[emitreduct['Column Number']==6].rename(columns={'Response Answer':'Base Emissions'})
emittgt = emitreduct[emitreduct['Column Number']==7].rename(columns={'Response Answer':'Reduction%'})
tgtyr = emitreduct[emitreduct['Column Number']==8].rename(columns={'Response Answer':'Year'})
tgtabs = emitreduct[emitreduct['Column Number']==9].rename(columns={'Response Answer':'Abs Tgt'})
tgt2day = emitreduct[emitreduct['Column Number']==10].rename(columns={'Response Answer':'Tgt2day'})
emitbase = pd.merge(emitreductsec, emitbase, on=['Account Number', 'Row Number'])[['Account Number', 'Sector','Row Number', 'Base Emissions']]
emittgt = pd.merge(emitbase, emittgt, on=['Account Number', 'Row Number'])[['Account Number','Row Number', 'Sector' ,'Base Emissions', 'Reduction%']]
emittgt = pd.merge(emittgt, emitbaseyr, on=['Account Number', 'Row Number'])[['Account Number','Row Number', 'Sector' ,'Base Emissions', 'Base Year','Reduction%']]
emittgt = pd.merge(emittgt, tgtyr, on=['Account Number', 'Row Number'])[['Account Number','Row Number', 'Sector' ,'Base Emissions', 'Base Year','Reduction%', 'Year']]
emittgt = pd.merge(emittgt, tgtabs, on=['Account Number', 'Row Number'])[['Account Number','Row Number', 'Sector' ,'Base Emissions', 'Base Year','Reduction%', 'Abs Tgt', 'Year']]
emittgt = pd.merge(emittgt, tgt2day, on=['Account Number', 'Row Number'])[['Account Number','Row Number', 'Sector' ,'Base Emissions', 'Base Year','Reduction%', 'Abs Tgt', 'Year','Tgt2day']]
emittgt[['Base Emissions', 'Reduction%','Abs Tgt','Tgt2day']]=emittgt[['Base Emissions', 'Reduction%','Abs Tgt','Tgt2day']].astype('float')
emittgt[['Year', 'Base Year']]=emittgt[['Year', 'Base Year']].astype('float')

We can calculate a GHG Reduction Commitment Score which takes into account the urgency with which cities have promised to act on climate change. It measures the rate at which cities are promising to reduce emissions.

# Commitment Score = $ \frac{Emissions \enspace Reduction \%}{No \enspace of\enspace Years} $

We can also calculate a Proportional Commitment Score, which takes into account the current level of emissions per capita (High, Medium, Low), so that cities with high emissions need to commit to a greater rate of reduction to have the same score as cities that are already low on emissions.

# Proportional Commitment Score = $ \frac{Emissions \enspace Reduction \%*Emissions\enspace Penalty}{Number\enspace of\enspace Years} $


We can also calculate a performance score, comparing the declared reduction rate so far with respect to committed reduction rate over the entire period of commitment. Cities that are on-track or doing better than committed reductions have a higher score.

# Performance Score = $ \frac{Actual \enspace Emissions \enspace Reduction \enspace Rate}{Committed\enspace Reduction\enspace Rate} $

In [ ]:
pmap = {'Low':1, 'Medium':0.9, 'High':0.8}
emittgt = pd.merge(emittgt, population, on='Account Number')
emittgt['Emissions pc'] = emittgt['Base Emissions']/emittgt['Popn'].astype('float')
emittgt['Emissions pc'] = pd.qcut(emittgt['Emissions pc'],3,labels=['Low','Medium','High'])
emittgt['Penalty'] = emittgt['Emissions pc'].map(pmap).astype('float')
emittgt['CS'] = (emittgt['Reduction%'])/(emittgt['Year']-emittgt['Base Year']).values
emittgt['PCS'] = (emittgt['Reduction%']* emittgt['Penalty'])/(emittgt['Year']-emittgt['Base Year']).values
emittgt['PS'] = (emittgt['Tgt2day']/(2020-emittgt['Base Year']))*emittgt['Penalty']/ emittgt['CS']
emittgt['Org']=emittgt['Account Number'].map(acctorg)
emittgt = emittgt.replace([np.inf, -np.inf], np.nan)
emittgt = emittgt.dropna()
fig, ax = plt.subplots(1,2, figsize=(15,5), constrained_layout=True)
emittgt = emittgt.sort_values(by='PCS', ascending=False)
ax[0].barh(emittgt.head(10)['Org'], emittgt.head(10)['CS'])
ax[0].invert_yaxis()
emittgt = emittgt.sort_values(by='PS', ascending=False)
ax[1].barh(emittgt.head(10)['Org'], emittgt.head(10)['PS'])
ax[0].set_title('Top Cities by Commitment', fontsize=20)
ax[1].set_title('Best Tracking Cities', fontsize=20)
ax[1].invert_yaxis()
for ax in [ax[0], ax[1]]:
    ax.xaxis.grid(True, which='major', linestyle='-', linewidth=0.15)
    ax.yaxis.grid(True, which='major', linestyle='-', linewidth=0.1)
    ax.tick_params(axis='y', which='major', labelsize=15)
